In [ ]:
!pip install -r requirements.txt

In [1]:
import cv2
from ultralytics import YOLO

## Check for GPU availability

In [2]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA RTX A4000


In [6]:
!nvcc --version
print("\npytorch version:", torch.__version__)

/bin/bash: /home/servicer/miniconda3/envs/peds_tracker_gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0

pytorch version: 2.0.0


## Initialize CPU Device

In [7]:
device = torch.device("cpu")
print("using", device, "device")

using cpu device


## Initialize the YOLOv9 model

In [8]:
# set to desired gpu number
# torch.cuda.set_device(0)

# specify use of gpu in model
model = YOLO('yolov9e-seg.pt')
model.to(device)

YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Silence()
      (1): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (3): RepNCSPELAN4(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Sequential(
          (0): RepCSP(
            (cv1): Conv(
              (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bi

## Initialize video source

In [12]:
# Define video source
vid_path = "./sources/two_then_one_ppl.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

Video opened successfully!
Video Frame Rate: 30.0 fps


## Define YOLO classes

In [9]:
PERSON = 0
BIKE = 1
CAR = 2
MOTORCYCLE = 3
BUS = 5

## Process video source

In [14]:
ped_times = {}

while vid_cap.isOpened():
	success, frame = vid_cap.read()
	if not success:
		break

	# Perform detection and tracking
	results = model.track(frame, persist=True, conf=0.2, classes=[PERSON, BIKE], iou=0.5, show=False, tracker="bytetrack.yaml")

	# Print frame
	frame_num = int(vid_cap.get(cv2.CAP_PROP_POS_FRAMES))

	# Get annotated frame and display it
	# annotated_frame = results[0].plot()
	# cv2.imshow('YOLOv9 Tracking', annotated_frame)
	for i, r in enumerate(results):
		for index, box in enumerate(r.boxes):
			# id of object
			print(box.id)
			print(box.cls)
			if(box.id and box.cls[0] != None):
				# print(box.id)
				# print(box.cls)
				tracker_id = int(box.id)
				tracker_cls = int(box.cls[0])
				print("class:", tracker_cls)
				print("id", tracker_id)

				obj_idx = "class_" + str(tracker_cls) + "_idx_" + str(tracker_id)
				if(not(obj_idx in ped_times)):
					ped_times[obj_idx] = frame_num / vid_fps
    
	print("frame:", frame_num)

print(ped_times)

	# # Press 'q' to exit
	# if cv2.waitKey(1) & 0xFF == ord('q'):
	# 	break


0: 384x640 2 persons, 1052.0ms
Speed: 38.4ms preprocess, 1052.0ms inference, 106.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 1

0: 384x640 2 persons, 949.7ms
Speed: 1.9ms preprocess, 949.7ms inference, 25.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 2

0: 384x640 2 persons, 1057.2ms
Speed: 1.6ms preprocess, 1057.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 3

0: 384x640 2 persons, 1009.3ms
Speed: 1.6ms preprocess, 1009.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 4

0: 384x640 2 persons, 935.4ms
Speed: 1.8ms preprocess, 935.4ms inference, 18.0ms postprocess per image at shape (1, 3

## Process and Display video source

In [10]:
# Define video source
vid_path = "./sources/2024-05-10T08-45-36.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

# Specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# Use gpu for model
model.to(device)
torch.cuda.synchronize()

# Read and process the video
while vid_cap.isOpened():

    # Get the next captured frame
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform detection and tracking on frame
    results = model.track(
        frame,
        persist=True,
        conf=0.2,
        iou=0.5,
        show=False,
        tracker="bytetrack.yaml",
        classes=[PERSON]
    )

    # Get annotated frame and display it
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv9 Tracking', annotated_frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
vid_cap.release()
cv2.destroyAllWindows()

Video opened successfully!
Video Frame Rate: 30.0 fps

0: 384x640 (no detections), 969.1ms
Speed: 3.0ms preprocess, 969.1ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)



QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is not the object's thread (0x340aea40).
Cannot move to target thread (0x33f94a00)

QObject::moveToThread: Current thread (0x33f94a00) is n

0: 384x640 (no detections), 960.4ms
Speed: 12.9ms preprocess, 960.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 843.5ms
Speed: 13.6ms preprocess, 843.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 900.1ms
Speed: 11.0ms preprocess, 900.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 697.3ms
Speed: 2.9ms preprocess, 697.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 698.2ms
Speed: 3.1ms preprocess, 698.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 764.7ms
Speed: 5.7ms preprocess, 764.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1150.9ms
Speed: 2.7ms preprocess, 1150.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 836.8ms
Speed: 2.9ms p

KeyboardInterrupt: 

In [11]:
# Release resources
vid_cap.release()
cv2.destroyAllWindows()